In [3]:
!pip install ndjson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ndjson-0.3.1-py2.py3-none-any.whl (5.3 kB)


In [19]:
import ndjson
import os
from tqdm.auto import tqdm
import urllib.request
import glob
import numpy as np

In [15]:
print('Loading data')

if not os.path.exists("categories.txt"):
    os.system("wget 'https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt'")


f = open("categories.txt","r")
# And for reading use
classes = f.readlines()
f.close()

classes = [c.replace('\n','').replace(' ','_') for c in classes]

def download():
    base = 'https://storage.googleapis.com/quickdraw_dataset/full/simplified/'
    
    for c in tqdm(classes):        
        cls_url = c.replace('_', '%20')
        path = base+cls_url+'.ndjson'
        # print(path)
        urllib.request.urlretrieve(path, 'scribble_data_ndjson/'+c+'.ndjson')

if not os.path.exists('scribble_data_ndjson'):
    os.makedirs('scribble_data_ndjson')
    download()
else:
    print('scribble_data_ndjson already exists')

Loading data


  0%|          | 0/345 [00:00<?, ?it/s]

In [32]:
max_items_per_class = 20
def load_data_for_diffusion(root, max_items_per_class= max_items_per_class ):
    all_files = glob.glob(os.path.join(root, '*.ndjson'))

    #initialize variables
    imgs = np.empty([0, 255*255])
    labels = []

    for idx, file in enumerate(all_files):
        with open(file) as f:
          data = ndjson.load(f)
          data_np = np.array(data)
          print(type(data))
          print(len(data))
          print(data_np.shape)
        
        break
  
        data = data[0: max_items_per_class, :]

        class_name, ext = os.path.splitext(os.path.basename(file))
        labels.extend(["a scribble of " + class_name for i in range(data.shape[0])])

        imgs = np.concatenate((imgs, data), axis=0)


    return imgs, labels, data

imgs, labels, data = load_data_for_diffusion('scribble_data_ndjson')


<class 'list'>
142167
(142167,)


In [39]:
from PIL import Image
print(data[0])
drawing = data[0]['drawing']
drawning = np.array(drawing)


{'word': 'mushroom', 'countrycode': 'US', 'timestamp': '2017-01-24 16:43:19.59937 UTC', 'recognized': True, 'key_id': '5340017608097792', 'drawing': [[[128, 124, 86, 49, 26, 17, 4, 0, 2, 10, 33, 48, 83, 139, 157, 183, 193, 202, 207, 207, 178, 173, 173, 197, 199, 196], [239, 145, 139, 123, 109, 101, 82, 49, 39, 27, 11, 5, 0, 3, 7, 22, 31, 48, 76, 104, 139, 167, 185, 244, 255, 254]]]}
